In [1]:
import pickle
import qdrant_client
import transformers 
import torch

c:\Users\prati\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from qdrant_client.http import models

from transformers import pipeline

from sentence_transformers import SentenceTransformer



In [3]:
import pandas as pd 
dataset = pd.read_csv('bigBasketProducts.csv')

In [4]:
dataset['page_content'] =  "product name : "+ dataset['product'] + " Category :" +dataset['category'] + " Sub_Category : "+ dataset['sub_category'] + " Brand :"+ dataset['brand'] + " Type :"+ dataset['type']+ " Market_price :"+dataset['market_price'].astype(str) + " Sale_price:"+ dataset['sale_price'].astype(str) + " Rating :"+dataset['rating'].astype(str)+" Description : " + dataset['description'] 
dataset.shape
dataset.dropna(inplace = True)
print(dataset['page_content'].isnull().sum())

0


In [5]:
dataset['metadata'] = "product name : "+ dataset['product'] + " Category :" +dataset['category'] + " Sub_Category : "+ dataset['sub_category'] + " Brand :"+ dataset['brand'] + " Type :"+ dataset['type']+ " Market_price :"+dataset['market_price'].astype(str) + " Sale_price:"+ dataset['sale_price'].astype(str) + " Rating :"+dataset['rating'].astype(str)

In [6]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

collection_name = "qdrant_db"

collection = client.create_collection(collection_name=collection_name ,   vectors_config=models.VectorParams(
            size=768,
            distance=models.Distance.COSINE,
        ))


In [7]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("all-mpnet-base-v2", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [8]:
from tqdm.auto import tqdm
from typing import List



In [9]:
batch_size = 256  # specify batch size according to your RAM and compute, higher batch size = more RAM usage
dataset = dataset[['metadata' , 'page_content']]
for index in tqdm(range(0, len(dataset), batch_size)):
    i_end = min(index + batch_size, len(dataset))  # find end of batch
    batch = dataset.iloc[index:i_end]  # extract batch
    emb = retriever.encode(batch['page_content'].tolist()).tolist()  # generate embeddings for batch
    meta = batch.to_dict(orient='records')  # get metadata
    ids = list(range(index, i_end))  # create unique IDs

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=emb, payloads=meta),
    )

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(dataset)

100%|██████████| 74/74 [1:41:38<00:00, 82.41s/it] 

Vector count in collection: 18840


In [10]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
print(reader.model, reader)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [11]:
def get_relevant_context(question: str, top_k: int) -> List[str]:
    """
    Get the relevant context for a given question

    Args:
        question (str): What do we want to know?
        top_k (int): Top K results to return

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = retriever.encode(question).tolist()  # generate embeddings for the question

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [x.payload["metadata"], x.payload["page_content"]] for x in result
        ]  # extract title and payload from result
        return context

    except Exception as e:
        print({e})

In [12]:
def extract_answer(question: str, context: List[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c[1])

        # add the context to answer dict for printing both together, we print only first 500 characters of plot
        answer["Product"] = c[0]
        results.append(answer)

    # sort the result based on the score from reader model
    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        print(f"{i+1}", end=" ")
        print(
            "Answer: ",
            sorted_result[i]["answer"],
            "\n  Product: ",
            sorted_result[i]["Product"],
            "\n  score: ",
            sorted_result[i]["score"],
        )


question = "Best oil for hair"
context = get_relevant_context(question, top_k=1)
extract_answer(question=question , context= context)

1 Answer:  Bajaj Almond Drops is one of the best hair oils for your hair 
  Product:  product name : Hair Oil - Almond Drops Category :Beauty & Hygiene Sub_Category : Hair Care Brand :Bajaj Type :Hair Oil & Serum Market_price :290.0 Sale_price:261.0 Rating :4.4 
  score:  0.06513652205467224


In [17]:
import json 

qdrant_collection_path = "collection.pkl"
with open(qdrant_collection_path, 'wb') as file:
 pickle.dump(client , file)

embedding_model_json = {
    "architecture": str(retriever),
    "state_dict": retriever.state_dict()
}

# Save PyTorch Model JSON to File
embedding_model_path = "embedding_model.pkl"
with open(embedding_model_path, 'wb') as file:
    pickle.dump(retriever, file)

with open("reader.pkl", 'wb') as file:
    pickle.dump(reader, file)
